In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio


2025-02-23 00:29:32.876464: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 00:29:32.884225: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-23 00:29:32.895249: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-23 00:29:32.895268: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 00:29:32.902725: I tensorflow/core/platform/cpu_feature_gua

In [2]:
import utilities as util

In [3]:
# Create a dataset from a csv file

csv_file = "dataClassified/data.csv"

# csv is in format: filename, label

prepend_path = "dataClassified/"



In [4]:
# Load datasets from util.Dataset

dataset_train = util.CustomDataset(csv_file, prepend_path, "train")
dataset_val = util.CustomDataset(csv_file, prepend_path, "val")
dataset_test = util.CustomDataset(csv_file, prepend_path, "test")

2025-02-23 00:29:37.300166: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-23 00:29:37.326830: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)


In [6]:
import finetuning_utils as ftu

In [7]:
embeds, labels = ftu.preprocess_dataset(dataset_train, yamnet_model)


2025-02-23 00:30:09.771917: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
embeds_val, labels_val = ftu.preprocess_dataset(dataset_val, yamnet_model)

2025-02-23 00:30:15.316946: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [9]:
embeds_test, labels_test = ftu.preprocess_dataset(dataset_test, yamnet_model)

2025-02-23 00:30:18.767384: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define the model
model = tf.keras.Sequential([
    layers.Input(shape=(1024,)),  # YAMNet embeddings are 1024-dimensional
    layers.Dense(512, activation='relu'),

    layers.Dense(128, activation='relu'),
    layers.Dense(util.num_classes, activation='softmax')
])


from tensorflow.keras.optimizers import Adam

# Optimizer: Adam with tuned learning rate
optimizer = Adam(learning_rate=1e-4 , weight_decay=1e-5)


# Compile the model
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)


In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np

# Number of folds
k_folds = 5

# Prepare KFold splitter
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Arrays to store fold results
fold_accuracies = []

# Convert labels to numpy if not already
labels_np = np.array(labels)
embeds_np = np.array(embeds)

# K-Fold Cross Validation (using separate validation set)
for fold, (train_idx, _) in enumerate(kf.split(embeds_np)):
    print(f"\n🔄 Training fold {fold + 1}/{k_folds}...")

    # Split data for training (validation is separate)
    X_train, y_train = embeds_np[train_idx], labels_np[train_idx]

    # Train the model
    history = model.fit(X_train, y_train,
                        validation_data=(embeds_val, labels_val),  # Dedicated validation set
                        epochs=50,
                        batch_size=32,
                        callbacks=[early_stop, reduce_lr],
                        verbose=1,
    )
    #                    class_weight=class_weights)

    # Evaluate on the dedicated validation set
    val_preds = model.predict(embeds_val)
    val_preds_classes = np.argmax(val_preds, axis=1)
    val_accuracy = accuracy_score(labels_val, val_preds_classes)
    fold_accuracies.append(val_accuracy)
    print(f"✅ Fold {fold + 1} Validation Accuracy: {val_accuracy:.4f}")

# Final Cross-Validation Result
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)
print(f"\n🎯 Final Validation Accuracy (across folds): {mean_accuracy:.4f} ± {std_accuracy:.4f}")



🔄 Training fold 1/5...
Epoch 1/50
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7562 - loss: 0.6094 - val_accuracy: 0.7690 - val_loss: 0.5490 - learning_rate: 1.0000e-04
Epoch 2/50
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8112 - loss: 0.4722 - val_accuracy: 0.7811 - val_loss: 0.5382 - learning_rate: 1.0000e-04
Epoch 3/50
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8211 - loss: 0.4440 - val_accuracy: 0.7701 - val_loss: 0.5587 - learning_rate: 1.0000e-04
Epoch 4/50
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8303 - loss: 0.4196 - val_accuracy: 0.7793 - val_loss: 0.5482 - learning_rate: 1.0000e-04
Epoch 5/50
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8453 - loss: 0.3857 - val_accuracy: 0.7768 - val_loss: 0.5531 - learning_rate: 1.0000e-04
Epoch 6/50
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8573 - loss: 0.3646 - val_accuracy: 0.7784 - val_loss: 0.5576 - learning_rate: 5.0000e-05
Epoch 7/50
1039/1039 ━━━━━━━

In [40]:
# Check if tensorflow is using GPU

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [38]:
# Check the model performance on the test set

test_loss, test_acc = model.evaluate(embeds_test, labels_test, verbose=2)
print('\nTest accuracy:', test_acc)

175/175 - 0s - 748us/step - accuracy: 0.8356 - loss: 0.4504

Test accuracy: 0.8355674743652344
